# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd700284dc0>
├── 'a' --> tensor([[-0.8330, -1.5985,  0.4937],
│                   [-0.5222,  1.2119, -0.0678]])
└── 'x' --> <FastTreeValue 0x7fd700284ca0>
    └── 'c' --> tensor([[ 0.4604, -1.1836,  1.5184,  0.6800],
                        [ 0.0513,  1.6735,  0.6170,  0.7493],
                        [ 0.4952, -0.2171, -0.4603,  1.5569]])

In [4]:
t.a

tensor([[-0.8330, -1.5985,  0.4937],
        [-0.5222,  1.2119, -0.0678]])

In [5]:
%timeit t.a

65.4 ns ± 0.0589 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd700284dc0>
├── 'a' --> tensor([[ 0.1341,  0.5611, -0.3961],
│                   [ 0.1902, -1.9328,  0.3654]])
└── 'x' --> <FastTreeValue 0x7fd700284ca0>
    └── 'c' --> tensor([[ 0.4604, -1.1836,  1.5184,  0.6800],
                        [ 0.0513,  1.6735,  0.6170,  0.7493],
                        [ 0.4952, -0.2171, -0.4603,  1.5569]])

In [7]:
%timeit t.a = new_value

68.5 ns ± 0.0638 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.8330, -1.5985,  0.4937],
               [-0.5222,  1.2119, -0.0678]]),
    x: Batch(
           c: tensor([[ 0.4604, -1.1836,  1.5184,  0.6800],
                      [ 0.0513,  1.6735,  0.6170,  0.7493],
                      [ 0.4952, -0.2171, -0.4603,  1.5569]]),
       ),
)

In [10]:
b.a

tensor([[-0.8330, -1.5985,  0.4937],
        [-0.5222,  1.2119, -0.0678]])

In [11]:
%timeit b.a

58.4 ns ± 0.124 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.5538, -2.1823, -0.1172],
               [ 0.6849, -1.5255, -1.4571]]),
    x: Batch(
           c: tensor([[ 0.4604, -1.1836,  1.5184,  0.6800],
                      [ 0.0513,  1.6735,  0.6170,  0.7493],
                      [ 0.4952, -0.2171, -0.4603,  1.5569]]),
       ),
)

In [13]:
%timeit b.a = new_value

490 ns ± 0.123 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

830 ns ± 0.772 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 10.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

146 µs ± 413 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 1.09 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd64acc0160>
├── 'a' --> tensor([[[-0.8330, -1.5985,  0.4937],
│                    [-0.5222,  1.2119, -0.0678]],
│           
│                   [[-0.8330, -1.5985,  0.4937],
│                    [-0.5222,  1.2119, -0.0678]],
│           
│                   [[-0.8330, -1.5985,  0.4937],
│                    [-0.5222,  1.2119, -0.0678]],
│           
│                   [[-0.8330, -1.5985,  0.4937],
│                    [-0.5222,  1.2119, -0.0678]],
│           
│                   [[-0.8330, -1.5985,  0.4937],
│                    [-0.5222,  1.2119, -0.0678]],
│           
│                   [[-0.8330, -1.5985,  0.4937],
│                    [-0.5222,  1.2119, -0.0678]],
│           
│                   [[-0.8330, -1.5985,  0.4937],
│                    [-0.5222,  1.2119, -0.0678]],
│           
│                   [[-0.8330, -1.5985,  0.4937],
│                    [-0.5222,  1.2119, -0.0678]]])
└── 'x' --> <FastTreeValue 0x7fd64acc0100>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.4 µs ± 110 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd64acdd910>
├── 'a' --> tensor([[-0.8330, -1.5985,  0.4937],
│                   [-0.5222,  1.2119, -0.0678],
│                   [-0.8330, -1.5985,  0.4937],
│                   [-0.5222,  1.2119, -0.0678],
│                   [-0.8330, -1.5985,  0.4937],
│                   [-0.5222,  1.2119, -0.0678],
│                   [-0.8330, -1.5985,  0.4937],
│                   [-0.5222,  1.2119, -0.0678],
│                   [-0.8330, -1.5985,  0.4937],
│                   [-0.5222,  1.2119, -0.0678],
│                   [-0.8330, -1.5985,  0.4937],
│                   [-0.5222,  1.2119, -0.0678],
│                   [-0.8330, -1.5985,  0.4937],
│                   [-0.5222,  1.2119, -0.0678],
│                   [-0.8330, -1.5985,  0.4937],
│                   [-0.5222,  1.2119, -0.0678]])
└── 'x' --> <FastTreeValue 0x7fd700269f10>
    └── 'c' --> tensor([[ 0.4604, -1.1836,  1.5184,  0.6800],
                        [ 0.0513,  1.6735,  0.6170,  0.7493],
                 

In [23]:
%timeit t_cat(trees)

31.3 µs ± 55.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.7 µs ± 42.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.8330, -1.5985,  0.4937],
                [-0.5222,  1.2119, -0.0678]],
       
               [[-0.8330, -1.5985,  0.4937],
                [-0.5222,  1.2119, -0.0678]],
       
               [[-0.8330, -1.5985,  0.4937],
                [-0.5222,  1.2119, -0.0678]],
       
               [[-0.8330, -1.5985,  0.4937],
                [-0.5222,  1.2119, -0.0678]],
       
               [[-0.8330, -1.5985,  0.4937],
                [-0.5222,  1.2119, -0.0678]],
       
               [[-0.8330, -1.5985,  0.4937],
                [-0.5222,  1.2119, -0.0678]],
       
               [[-0.8330, -1.5985,  0.4937],
                [-0.5222,  1.2119, -0.0678]],
       
               [[-0.8330, -1.5985,  0.4937],
                [-0.5222,  1.2119, -0.0678]]]),
    x: Batch(
           c: tensor([[[ 0.4604, -1.1836,  1.5184,  0.6800],
                       [ 0.0513,  1.6735,  0.6170,  0.7493],
                       [ 0.4952, -0.2171, -0.4603,  1.5569]],
         

In [26]:
%timeit Batch.stack(batches)

79.4 µs ± 202 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.8330, -1.5985,  0.4937],
               [-0.5222,  1.2119, -0.0678],
               [-0.8330, -1.5985,  0.4937],
               [-0.5222,  1.2119, -0.0678],
               [-0.8330, -1.5985,  0.4937],
               [-0.5222,  1.2119, -0.0678],
               [-0.8330, -1.5985,  0.4937],
               [-0.5222,  1.2119, -0.0678],
               [-0.8330, -1.5985,  0.4937],
               [-0.5222,  1.2119, -0.0678],
               [-0.8330, -1.5985,  0.4937],
               [-0.5222,  1.2119, -0.0678],
               [-0.8330, -1.5985,  0.4937],
               [-0.5222,  1.2119, -0.0678],
               [-0.8330, -1.5985,  0.4937],
               [-0.5222,  1.2119, -0.0678]]),
    x: Batch(
           c: tensor([[ 0.4604, -1.1836,  1.5184,  0.6800],
                      [ 0.0513,  1.6735,  0.6170,  0.7493],
                      [ 0.4952, -0.2171, -0.4603,  1.5569],
                      [ 0.4604, -1.1836,  1.5184,  0.6800],
                      [ 0.0513,  

In [28]:
%timeit Batch.cat(batches)

145 µs ± 119 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

336 µs ± 2.53 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
